<a href="https://colab.research.google.com/github/backlashblitz/Bangla-Book-Recommendation-Dataset/blob/main/colabnotebooks/LightGCN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGCN
**Bangla Book Recommendation Dataset**

▶️ **Just click `Runtime → Run all` to get started!**

In [ ]:
import os, shutil
from huggingface_hub import hf_hub_download

os.system("pip install -q huggingface_hub tqdm")

REPO_ID = "DevnilMaster1/Bangla-Book-Recommendation-Dataset"
DATA_FOLDER = "RokomariBG_Dataset"
os.makedirs(DATA_FOLDER, exist_ok=True)

FILES_NEEDED = ['user_to_review.json', 'book_to_review.json']

for filename in FILES_NEEDED:
    dest = os.path.join(DATA_FOLDER, filename)
    if not os.path.exists(dest):
        try:
            downloaded_path = hf_hub_download(
                repo_id=REPO_ID,
                filename=f"{DATA_FOLDER}/{filename}",
                repo_type="dataset"
            )
            shutil.copy(downloaded_path, dest)
            print(f"✅ Saved: {filename}")
        except Exception as e:
            print(f"⚠️ Error downloading {filename}: {e}")
    else:
        print(f"✅ Already present: {filename}")

user_to_review_path = os.path.join(DATA_FOLDER, "user_to_review.json")
book_to_review_path = os.path.join(DATA_FOLDER, "book_to_review.json")

print("\n🎉 Environment Ready!")

✅ Already present: user_to_review.json
✅ Already present: book_to_review.json

🎉 Environment Ready!


In [ ]:
import numpy as np
from typing import Dict, List, Set

class RankingEvaluator:
    def __init__(self):
        pass

    def hit_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        top_k = predictions[:k]
        return 1.0 if any(item in ground_truth for item in top_k) else 0.0

    def mrr_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        top_k = predictions[:k]
        for rank, item in enumerate(top_k, start=1):
            if item in ground_truth:
                return 1.0 / rank
        return 0.0

    def dcg_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        dcg = 0.0
        top_k = predictions[:k]
        for rank, item in enumerate(top_k, start=1):
            if item in ground_truth:
                dcg += 1.0 / np.log2(rank + 1)
        return dcg

    def idcg_at_k(self, ground_truth: Set[str], k: int) -> float:
        ideal_k = min(len(ground_truth), k)
        idcg = sum(1.0 / np.log2(rank + 1) for rank in range(1, ideal_k + 1))
        return idcg

    def ndcg_at_k(self, predictions: List[str], ground_truth: Set[str], k: int) -> float:
        dcg = self.dcg_at_k(predictions, ground_truth, k)
        idcg = self.idcg_at_k(ground_truth, k)
        if idcg == 0.0:
            return 0.0
        return dcg / idcg

    def evaluate(self, predictions: Dict[str, List[str]], ground_truth: Dict[str, Set[str]]) -> Dict[str, float]:
        metrics = {
            'Hit@5': [],
            'Hit@10': [],
            'Hit@50': [],
            'MRR@10': [],
            'NDCG@10': [],
            'NDCG@50': []
        }
        common_users = set(predictions.keys()) & set(ground_truth.keys())
        for user_id in common_users:
            preds = predictions[user_id]
            gt = ground_truth[user_id]
            if len(gt) == 0:
                continue
            metrics['Hit@5'].append(self.hit_at_k(preds, gt, 5))
            metrics['Hit@10'].append(self.hit_at_k(preds, gt, 10))
            metrics['Hit@50'].append(self.hit_at_k(preds, gt, 50))
            metrics['MRR@10'].append(self.mrr_at_k(preds, gt, 10))
            metrics['NDCG@10'].append(self.ndcg_at_k(preds, gt, 10))
            metrics['NDCG@50'].append(self.ndcg_at_k(preds, gt, 50))
        results = {}
        for metric_name, values in metrics.items():
            results[metric_name] = np.mean(values) if len(values) > 0 else 0.0
        return results

In [ ]:
import json
import random
from collections import defaultdict
from typing import Dict, List, Set
from tqdm.auto import tqdm
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

class BaseRecommender:
    def fit(self):
        raise NotImplementedError
    def recommend(self, user_id: str, k: int = 10) -> List[str]:
        raise NotImplementedError

class LightGCNRecommender(BaseRecommender):
    def __init__(
        self,
        train_user_book: Dict[str, List[str]],
        n_factors: int = 64,
        n_layers: int = 3,
        n_epochs: int = 10,
        lr: float = 0.01,
        reg: float = 1e-4,
        n_neg: int = 1,
        seed: int = 42,
        device: str = None,
        batch_size: int = 4096,
        neg_per_user: int = 1,
    ):
        self.train_user_book = {u: list(set(b)) for u, b in train_user_book.items()}
        self.n_factors = n_factors
        self.n_layers = n_layers
        self.n_epochs = n_epochs
        self.lr = lr
        self.reg = reg
        self.n_neg = n_neg
        self.rng = np.random.default_rng(seed)
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.user_ids = []
        self.book_ids = []
        self.user_id_map = {}
        self.book_id_map = {}
        self.n_users = 0
        self.n_items = 0
        self.A_hat = None
        self.user_emb = None
        self.item_emb = None
        self.final_user_emb = None
        self.final_item_emb = None
        self.batch_size = batch_size
        self.neg_per_user = neg_per_user

    def _build_sparse_graph(self):
        print("Building sparse LightGCN graph...")
        self.user_ids = sorted(self.train_user_book.keys())
        self.book_ids = sorted({b for books in self.train_user_book.values() for b in books})
        self.user_id_map = {u: i for i, u in enumerate(self.user_ids)}
        self.book_id_map = {b: i for i, b in enumerate(self.book_ids)}
        U, I = len(self.user_ids), len(self.book_ids)
        N = U + I
        rows, cols = [], []
        for u, books in self.train_user_book.items():
            uidx = self.user_id_map[u]
            for b in books:
                iidx = self.book_id_map[b]
                rows.append(uidx); cols.append(U + iidx)
                rows.append(U + iidx); cols.append(uidx)
        idx = torch.tensor([rows, cols], dtype=torch.long, device=self.device)
        val = torch.ones(idx.shape[1], dtype=torch.float32, device=self.device)
        A = torch.sparse_coo_tensor(idx, val, size=(N, N), device=self.device).coalesce()
        deg = torch.sparse.sum(A, dim=1).to_dense()
        deg_inv_sqrt = torch.pow(deg, -0.5)
        deg_inv_sqrt[torch.isinf(deg_inv_sqrt)] = 0.0
        r, c = A.indices()
        v_norm = A.values() * deg_inv_sqrt[r] * deg_inv_sqrt[c]
        self.A_hat = torch.sparse_coo_tensor(torch.stack([r, c], dim=0), v_norm, size=(N, N), device=self.device).coalesce()
        self.n_users, self.n_items = U, I

    def _propagate(self):
        all_emb0 = torch.cat([self.user_emb.weight, self.item_emb.weight], dim=0)
        embs = [all_emb0]
        x = all_emb0
        for _ in range(self.n_layers):
            x = torch.sparse.mm(self.A_hat, x)
            embs.append(x)
        all_emb = torch.mean(torch.stack(embs, dim=0), dim=0)
        return all_emb[: self.n_users], all_emb[self.n_users :]

    def fit(self):
        print(f"Training LightGCN on {self.device}...")
        self._build_sparse_graph()
        self.user_emb = nn.Embedding(self.n_users, self.n_factors).to(self.device)
        self.item_emb = nn.Embedding(self.n_items, self.n_factors).to(self.device)
        nn.init.xavier_uniform_(self.user_emb.weight)
        nn.init.xavier_uniform_(self.item_emb.weight)
        optimizer = optim.Adam(list(self.user_emb.parameters()) + list(self.item_emb.parameters()), lr=self.lr)
        user_pos_items = {self.user_id_map[u]: set(self.book_id_map[b] for b in books if b in self.book_id_map) for u, books in self.train_user_book.items() if u in self.user_id_map}
        users_list = np.array(list(user_pos_items.keys()), dtype=np.int64)
        for epoch in range(1, self.n_epochs + 1):
            self.rng.shuffle(users_list)
            total_loss, n_steps = 0.0, 0
            pbar = tqdm(range(0, len(users_list), self.batch_size), desc=f"LightGCN Epoch {epoch}")
            for start in pbar:
                batch_users = users_list[start : start + self.batch_size]
                if len(batch_users) == 0: continue
                pos_items = [int(self.rng.choice(list(user_pos_items[int(u)]))) for u in batch_users]
                neg_items = np.empty((len(batch_users), self.neg_per_user), dtype=np.int64)
                for i, u in enumerate(batch_users):
                    pos_set = user_pos_items[int(u)]
                    for j in range(self.neg_per_user):
                        neg = int(self.rng.integers(0, self.n_items))
                        while neg in pos_set: neg = int(self.rng.integers(0, self.n_items))
                        neg_items[i, j] = neg
                batch_users_t = torch.tensor(batch_users, device=self.device, dtype=torch.long)
                pos_items_t = torch.tensor(pos_items, device=self.device, dtype=torch.long)
                neg_items_t = torch.tensor(neg_items, device=self.device, dtype=torch.long)
                u_f, i_f = self._propagate()
                uvec, pvec, nvec = u_f[batch_users_t], i_f[pos_items_t], i_f[neg_items_t]
                pos_scores = torch.sum(uvec * pvec, dim=1, keepdim=True)
                neg_scores = torch.sum(uvec.unsqueeze(1) * nvec, dim=2)
                bpr = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores) + 1e-12))
                reg_val = self.reg * (self.user_emb(batch_users_t).norm(2).pow(2) + self.item_emb(pos_items_t).norm(2).pow(2) + self.item_emb(neg_items_t.view(-1)).norm(2).pow(2)) / len(batch_users)
                loss = bpr + reg_val
                optimizer.zero_grad(set_to_none=True)
                loss.backward(); optimizer.step()
                total_loss += float(loss.item()); n_steps += 1
                pbar.set_postfix(loss=f"{total_loss / n_steps:.5f}")
            print(f"Epoch {epoch}: avg_loss={total_loss / max(n_steps,1):.6f}")
        with torch.no_grad(): self.final_user_emb, self.final_item_emb = self._propagate()

    def recommend(self, user_id: str, k: int = 10) -> List[str]:
        if user_id not in self.user_id_map: return []
        uidx = self.user_id_map[user_id]
        with torch.no_grad():
            scores = torch.matmul(self.final_item_emb, self.final_user_emb[uidx]).cpu().numpy()
        seen = [self.book_id_map[b] for b in self.train_user_book[user_id] if b in self.book_id_map]
        if seen: scores[seen] = -1e9
        topk = np.argpartition(scores, -k)[-k:]
        return [self.book_ids[i] for i in topk[np.argsort(scores[topk])[::-1]]]

def load_json(path: str):
    with open(path, "r", encoding="utf-8") as f: return json.load(f)

def build_user_book_interactions(u_path, b_path):
    u_to_r, b_to_r = load_json(u_path), load_json(b_path)
    r_to_u = {str(x["review_id"]): str(x["user_id"]) for x in u_to_r}
    r_to_b = {str(x["review_id"]): str(x["book_id"]) for x in b_to_r}
    ub = defaultdict(list)
    for rid, uid in r_to_u.items():
        if rid in r_to_b: ub[uid].append(r_to_b[rid])
    return ub

def split_user_interactions(ub, seed=42):
    random.seed(seed)
    tr, va, te = {}, {}, {}
    for u, b in ub.items():
        b = list(set(b)); random.shuffle(b); n = len(b)
        if n < 3: tr[u], va[u], te[u] = b, [], []; continue
        nt, nv = int(0.7*n), int(0.15*n)
        tr[u], va[u], te[u] = b[:nt], b[nt:nt+nv], b[nt+nv:]
    return tr, va, te

if __name__ == "__main__":
    ub = build_user_book_interactions(user_to_review_path, book_to_review_path)
    tr, va, te = split_user_interactions(ub)
    gt = {u: set(b) for u, b in te.items() if b}
    model = LightGCNRecommender(tr, n_factors=64, n_layers=2, n_epochs=10)
    model.fit()
    preds = {u: model.recommend(u, 10) for u in tqdm(gt.keys(), desc="Predicting")}
    metrics = RankingEvaluator().evaluate(preds, gt)
    print("\n===== LightGCN Results =====")
    for m, v in metrics.items(): print(f"{m}: {v:.4f}")

Training LightGCN on cuda...
Building sparse LightGCN graph...
Epoch 10: avg_loss=0.018455

===== LightGCN Results =====
Hit@5: 0.1831
Hit@10: 0.2218
Hit@50: 0.2218
MRR@10: 0.1355
NDCG@10: 0.1237
NDCG@50: 0.1236
